---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [243]:
import pandas as pd
import numpy as np
doc = []
with open('./../Datasets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(-20)

0           03/25/93 Total time of visit (in minutes):\n
1                         6/18/85 Primary Care Doctor:\n
2      sshe plans to move as of 7/8/71 In-Home Servic...
3                  7 on 9/27/75 Audit C Score Current:\n
4      2/6/96 sleep studyPain Treatment Pain Level (N...
5                      .Per 7/06/79 Movement D/O note:\n
6      4, 5/18/78 Patient's thoughts about current su...
7      10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                           3/7/86 SOS-10 Total Score:\n
9               (4/10/71)Score-1Audit C Score Current:\n
10     (5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC...
11                         4/09/75 SOS-10 Total Score:\n
12     8/01/98 Communication with referring physician...
13     1/26/72 Communication with referring physician...
14     5/24/1990 CPT Code: 90792: With medical servic...
15     1/25/2011 CPT Code: 90792: With medical servic...
16           4/12/82 Total time of visit (in minutes):\n
17          1; 10/13/1976 Audit

In [247]:
df.iloc[476]

'1989 Family Psych History: Family History of Suicidal Behavior: Ideation/Threat(s)\n'

In [151]:
# Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
import re
df.str.findall(r'((?:\d{0,2})?(?: Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-. ]{0,2}(?:\d{0,2})?[, ]{0,2}(?:\d{0,4}))').iloc[250:260]

250                 [May 2005]
251            [December 1998]
252                [ Jan 2007]
253                 [Feb 2016]
254    [Augusta , August 1979]
255                 [Oct 2014]
256                 [Aug 1988]
257                 [Sep 2015]
258                 [Apr 1976]
259                 [Nov 1979]
dtype: object

In [323]:
# 04/20/2009; 04/20/09; 4/20/09; 4/3/09
# Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
# 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
# Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
# Feb 2009; Sep 2009; Oct 2010
# 6/2008; 12/2009
# 2009; 2010
df2 = df.str.extract(r'(?P<Dates>(?:(?:\d{1,2})[-/](?:\d{1,2}[-/])(?:\d{2,4})|(?:\d{1,2})?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]{0,2}(?:\d{0,2}(?:nd|th|st)?)?[, ]{0,2}\d{2,4}|\d{4}))')

In [324]:
df2["Dates"].replace({"Jan[a-z]*":"01","Feb[a-z]*":"02","Mar[a-z]*":"03","Apr[a-z]*":"04","May[a-z]*":"05","Jun[a-z]*":"06","Jul[a-z]*":"07","Aug[a-z]*":"08","Sep[a-z]*":"09","Oct[a-z]*":"10","Nov[a-z]*":"11","Dec[a-z]*":"12","[., -]{1,2}":"/"," ":"/"},inplace = True, regex=True)

In [321]:
df2["Dates"].replace({"^\d{4}":"9999"},inplace = True, regex=True)

In [325]:
df2["Dates"].unique()

array(['03/25/93', '6/18/85', '7/8/71', '9/27/75', '2/6/96', '7/06/79',
       '5/18/78', '10/24/89', '3/7/86', '4/10/71', '5/11/85', '4/09/75',
       '8/01/98', '1/26/72', '5/24/1990', '1/25/2011', '4/12/82',
       '10/13/1976', '4/24/98', '5/21/77', '7/21/98', '10/21/79',
       '3/03/90', '2/11/76', '07/25/1984', '4/13/82', '9/22/89',
       '9/02/76', '9/12/71', '10/24/86', '03/31/1985', '7/20/72',
       '4/12/87', '06/20/91', '5/12/2012', '3/15/83', '2/14/73',
       '5/24/88', '7/27/1986', '1/14/81', '7/29/75', '6/24/87', '8/14/94',
       '4/13/2002', '8/16/82', '2/15/1998', '7/15/91', '06/12/94',
       '9/17/84', '2/28/75', '11/22/75', '5/24/91', '6/13/92', '7/11/71',
       '12/26/86', '10/11/1987', '3/14/95', '12/01/73', '12/5/2010',
       '08/20/1982', '7/24/95', '8/06/83', '02/22/92', '6/28/87',
       '07/29/1994', '08/11/78', '10/29/91', '7/6/91', '1/21/87',
       '11/3/1985', '7/04/82', '4/13/89', '7/11/77', '4/12/74',
       '09/19/81', '9/6/79', '12/5/87', '01/05

In [327]:

def cleanDate(row):
    if row["Dates"].str.contains("\d{1,2}/\d{1,2}/\d{2}"):
        row["Dates"].str.replace("/\d{2}")

df2.apply(cleanDate, axis = 1)

03/25/93
6/18/85
7/8/71
9/27/75
2/6/96
7/06/79
5/18/78
10/24/89
3/7/86
4/10/71
5/11/85
4/09/75
8/01/98
1/26/72
5/24/1990
1/25/2011
4/12/82
10/13/1976
4/24/98
5/21/77
7/21/98
10/21/79
3/03/90
2/11/76
07/25/1984
4/13/82
9/22/89
9/02/76
9/12/71
10/24/86
03/31/1985
7/20/72
4/12/87
06/20/91
5/12/2012
3/15/83
2/14/73
5/24/88
7/27/1986
1/14/81
7/29/75
6/24/87
8/14/94
4/13/2002
8/16/82
2/15/1998
7/15/91
06/12/94
9/17/84
2/28/75
11/22/75
5/24/91
6/13/92
7/11/71
12/26/86
10/11/1987
3/14/95
12/01/73
12/5/2010
08/20/1982
7/24/95
8/06/83
02/22/92
6/28/87
07/29/1994
08/11/78
10/29/91
7/6/91
1/21/87
11/3/1985
7/04/82
4/13/89
7/11/77
4/12/74
09/19/81
9/6/79
12/5/87
01/05/1999
4/22/80
10/04/98
6/29/81
8/04/78
7/07/1974
09/14/2000
5/18/71
8/09/1981
6/05/93
8/9/97
12/8/82
8/26/89
10/13/95
4/19/91
04/08/2004
9/20/76
12/08/1990
4/11/1974
7/18/86
3/31/91
5/13/72
11/14/83
8/16/92
10/05/97
07/18/2002
9/22/82
2/24/74
2/03/78
2/11/2006
8/22/83
5/04/74
7/20/2011
6/17/95
6/10/72
10/16/82
12/15/92
12/8/97
4/05/89


0      None
1      None
2      None
3      None
4      None
5      None
6      None
7      None
8      None
9      None
10     None
11     None
12     None
13     None
14     None
15     None
16     None
17     None
18     None
19     None
20     None
21     None
22     None
23     None
24     None
25     None
26     None
27     None
28     None
29     None
       ... 
470    None
471    None
472    None
473    None
474    None
475    None
476    None
477    None
478    None
479    None
480    None
481    None
482    None
483    None
484    None
485    None
486    None
487    None
488    None
489    None
490    None
491    None
492    None
493    None
494    None
495    None
496    None
497    None
498    None
499    None
Length: 500, dtype: object

In [262]:
df3 = df.str.extract(r'(?P<Dates>(?:(?P<Month1>\d{1,2})[-/](?P<Day1>\d{1,2}[-/])(?P<Year1>\d{2,4})|(?P<Day2>\d{1,2})?(?P<Month2>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]{0,2}(?P<Day3>\d{0,2}(?:nd|th|st)?)?[, ]{0,2}(?P<Year2>\d{2,4})|(?P<Year3>[^-]?\d{4})))')

In [3]:
def date_sorter():
    
    # Your code here
    
    return # Your answer here